# Content

### 1. Import resources

### 2.  Data cleaning and consistency checks (Step 6)

### 3. Data profile (Step 7)

# 1. Import resources

In [152]:
# Import libs
import pandas as pd
import numpy as np
import os

import random

In [153]:
# Set a path variable for the import
path = r'C:\Users\zhoux\Documents\CF-Data Analyst Program\DA-Immersion\Achievement 6\11-2023 Berlin Housing Price Analysis'

In [154]:
# Import dataset
df = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'real_estate_listings_clean.csv'))

In [155]:
df.head()

,url,energy,heating,price,area,rooms,fee,zipcode,construction_year,level,price_per_area
0,https://www.immowelt.de/expose/29fhx5b,Öl,Zentralheizung,249000.0,64.00,3.0,3.57,13505.0,1971.0,2.0,3890.625000
1,https://www.immowelt.de/expose/29hzk5u,Gas,"Fußbodenheizung, offener",1295000.0,136.06,4.0,3.57,10405.0,1900.0,5.0,9517.859768
2,https://www.immowelt.de/expose/292m95u,Gas,Etagenheizung,770000.0,120.00,4.0,3.57,12161.0,1900.0,5.0,6416.666667
3,https://www.immowelt.de/expose/29sh55u,Gas,Zentralheizung,349000.0,158.00,5.0,3.57,12103.0,1956.0,1.0,2208.860759
4,https://www.immowelt.de/expose/29r5y5t,Gas,Etagenheizung,494990.0,126.00,4.0,3.57,12157.0,1909.0,3.0,3928.492063


# 2. Data cleaning and consistency checks (Step 6)

In [156]:
df.shape

(4942, 11)

#### 2.1 Check datatypes

In [157]:
# Look at the data type of each column
df.dtypes

url                   object
energy                object
heating               object
price                float64
area                 float64
rooms                float64
fee                  float64
zipcode              float64
construction_year    float64
level                float64
price_per_area       float64
dtype: object

In [159]:
# Check for mixed datatypes
for col in df.columns.tolist():
  weird = (df[[col]].map(type) != df[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df[weird]) > 0:
    print (col)

In [160]:
df.dtypes

url                   object
energy                object
heating               object
price                float64
area                 float64
rooms                float64
fee                  float64
zipcode              float64
construction_year    float64
level                float64
price_per_area       float64
dtype: object

There are no columns with mixed datatypes. Four columns are string(object), the rest are numeric.

#### 2.2 Check null values or na observations 

In [161]:
# Check null values
df.isnull().sum()

url                  0
energy               0
heating              0
price                0
area                 0
rooms                0
fee                  0
zipcode              0
construction_year    0
level                0
price_per_area       0
dtype: int64

In [162]:
df['energy'].value_counts(dropna = False)

energy
na                           2112
Gas                          1414
Fernwärme                     829
Öl                            413
Luft-/                         34
Fernwärme,                     28
Elektro,                       24
Fußbodenheizung,               24
Elektro                        15
Solar                          12
Blockheizkraftwerk,            11
Gas,                           10
Erdwärme                        5
Pellets                         3
Flüssiggas                      2
Blockheizkraftwerk              2
Holz,                           1
Niedrigenergiehaus              1
Erdwärme,                       1
Fußbodenheizung, offener        1
Name: count, dtype: int64

In [163]:
df['heating'].value_counts(dropna=False)

heating
na                           2112
Zentralheizung               1839
Etagenheizung                 366
Fußbodenheizung,              206
Fußbodenheizung               174
Heizungsart                    70
Kamin,                         34
Wärmepumpe                     30
Wasser-                        24
Kf                             17
Kamin                          15
Niedrigenergiehaus             12
Fußbodenheizung, offener       11
Etagenheizung, offener          8
Etagenheizung,                  6
Heizungsartoffener              4
Luft-/                          3
Wärmepumpe,                     3
Fertighaus                      3
Minergie zertifiziert           2
Ofen,                           1
Wärmepumpe, offener             1
Kohle,                          1
Name: count, dtype: int64

Even though there are no empty observations in 'heating' and 'energy', there are a large amount (2112 counts) of na values.

#### 2.3 Check duplicates

In [164]:
# Look for duplicates
df[df.duplicated()]

,url,energy,heating,price,area,rooms,fee,zipcode,construction_year,level,price_per_area


There are no duplicates.

#### 2.4 Remove useless columns

The column 'url' is not useful as this analysis aims to get an understanding of the Berlin housing market, not the actual listings. Plus, many of the links may have expired.

In [165]:
df_new = df.drop(['url'], axis = 1)

In [166]:
df_new.head()

,energy,heating,price,area,rooms,fee,zipcode,construction_year,level,price_per_area
0,Öl,Zentralheizung,249000.0,64.00,3.0,3.57,13505.0,1971.0,2.0,3890.625000
1,Gas,"Fußbodenheizung, offener",1295000.0,136.06,4.0,3.57,10405.0,1900.0,5.0,9517.859768
2,Gas,Etagenheizung,770000.0,120.00,4.0,3.57,12161.0,1900.0,5.0,6416.666667
3,Gas,Zentralheizung,349000.0,158.00,5.0,3.57,12103.0,1956.0,1.0,2208.860759
4,Gas,Etagenheizung,494990.0,126.00,4.0,3.57,12157.0,1909.0,3.0,3928.492063


#### 2.4 Check anomalies

In [167]:
# Look at the overall picture of all numeric columns
df_new.describe()

,price,area,rooms,fee,zipcode,construction_year,level,price_per_area
count,4.942000e+03,4942.000000,4942.000000,4942.000000,4942.000000,4942.000000,4942.000000,4942.000000
mean,5.654009e+05,85.067321,2.805544,5.891574,11954.461554,1944.344193,3.401255,6401.269284
std,6.107572e+05,58.315922,1.582906,121.004862,1628.058346,143.609751,29.137154,3183.956274
min,3.495000e+04,13.000000,1.000000,-3.500000,1099.000000,1.000000,1.000000,956.521739
25%,2.590000e+05,54.070000,2.000000,3.570000,10589.000000,1910.000000,1.000000,4537.537879
50%,3.899000e+05,72.740000,3.000000,3.570000,12161.000000,1956.000000,1.000000,5843.852267
75%,6.690000e+05,101.000000,3.000000,3.570000,13088.000000,1994.000000,3.000000,8063.468346
max,1.590000e+07,970.000000,26.000000,7996.000000,77933.000000,2026.000000,2023.000000,87739.070663


The stats shows extreme max values in all numeric categories. Some of the min values are also wrong - for example, the construction year can't be 1, and the fee can't be a minus value.

##### 2.4.1 Check max and min values

##### Column 'rooms'

In [168]:
df_new.sort_values(by='rooms', ascending=False).head(20)                                              

,energy,heating,price,area,rooms,fee,zipcode,construction_year,level,price_per_area
3004,na,na,2195800.0,821.00,26.0,3.57,10587.0,1905.0,5.0,2674.543240
1910,na,na,590000.0,518.00,24.0,7.14,4567.0,1966.0,1.0,1138.996139
672,na,na,1250000.0,500.00,23.0,7.14,12047.0,1956.0,1.0,2500.000000
3255,Gas,Etagenheizung,860000.0,637.22,22.0,3.57,13405.0,1956.0,4.0,1349.612379
3223,na,na,6382140.0,72.74,22.0,3.57,10179.0,1986.0,1.0,87739.070663
3067,Gas,Zentralheizung,881700.0,678.23,22.0,7.14,12487.0,1996.0,1.0,1300.001474
2085,Gas,Zentralheizung,2342000.0,517.18,20.0,3.57,10245.0,1900.0,1.0,4528.404037
2086,Gas,Zentralheizung,2342000.0,517.18,20.0,3.57,10245.0,1900.0,1.0,4528.404037
4766,Öl,Zentralheizung,1550000.0,558.06,18.0,3.57,13589.0,1964.0,1.0,2777.479124
2959,na,na,2500000.0,550.00,16.0,7.14,12557.0,1997.0,1.0,4545.454545


The 5th row has many anomaly values: too small of an area to have 22 rooms, and an extremely high price per area. Since it is just one row, I will remove this row.

In [169]:

df_new = df_new.sort_values(by='rooms', ascending=False).drop([3223])

In [170]:
# Check whether the row is dropped
df_new.sort_values(by='rooms', ascending=False).head(10)

,energy,heating,price,area,rooms,fee,zipcode,construction_year,level,price_per_area
3004,na,na,2195800.0,821.00,26.0,3.57,10587.0,1905.0,5.0,2674.543240
1910,na,na,590000.0,518.00,24.0,7.14,4567.0,1966.0,1.0,1138.996139
672,na,na,1250000.0,500.00,23.0,7.14,12047.0,1956.0,1.0,2500.000000
3255,Gas,Etagenheizung,860000.0,637.22,22.0,3.57,13405.0,1956.0,4.0,1349.612379
3067,Gas,Zentralheizung,881700.0,678.23,22.0,7.14,12487.0,1996.0,1.0,1300.001474
2085,Gas,Zentralheizung,2342000.0,517.18,20.0,3.57,10245.0,1900.0,1.0,4528.404037
2086,Gas,Zentralheizung,2342000.0,517.18,20.0,3.57,10245.0,1900.0,1.0,4528.404037
4766,Öl,Zentralheizung,1550000.0,558.06,18.0,3.57,13589.0,1964.0,1.0,2777.479124
2959,na,na,2500000.0,550.00,16.0,7.14,12557.0,1997.0,1.0,4545.454545
963,Gas,Zentralheizung,1020000.0,300.00,15.0,4.76,12109.0,1963.0,1.0,3400.000000


In [171]:
df_new.sort_values(by='rooms', ascending=True).head(10)  

,energy,heating,price,area,rooms,fee,zipcode,construction_year,level,price_per_area
1980,Fernwärme,Zentralheizung,199900.0,43.00,1.0,3.57,13357.0,2023.0,3.0,4648.837209
4543,Gas,Zentralheizung,129000.0,31.63,1.0,3.57,12357.0,1940.0,1.0,4078.406576
3619,na,na,196000.0,22.40,1.0,3.57,13088.0,2024.0,1.0,8750.000000
2268,Öl,Zentralheizung,182000.0,33.48,1.0,3.57,10589.0,1907.0,3.0,5436.081243
2269,Gas,Zentralheizung,100000.0,27.00,1.0,3.57,13059.0,1987.0,1.0,3703.703704
1764,na,na,197120.0,22.40,1.0,3.57,13088.0,2024.0,24.0,8800.000000
3343,na,na,208465.0,24.10,1.0,3.57,13088.0,2024.0,1.0,8650.000000
1762,na,na,208465.0,24.10,1.0,3.57,13088.0,2024.0,24.0,8650.000000
4743,na,na,180000.0,41.02,1.0,3.57,12435.0,1956.0,4.0,4388.103364
1057,na,na,425000.0,48.00,1.0,3.57,10623.0,1897.0,4.0,8854.166667


Based on my experience with these online housing websites, the first row is a fraudulent listing because it has a very large area matched with an impossibly low price. I will remove the first row. 

In [172]:
df_new = df_new.sort_values(by='rooms', ascending=True).drop([3530])

In [173]:
# Check whether the row is dropped
df_new.sort_values(by='rooms', ascending=True).head(10)

,energy,heating,price,area,rooms,fee,zipcode,construction_year,level,price_per_area
1980,Fernwärme,Zentralheizung,199900.0,43.00,1.0,3.57,13357.0,2023.0,3.0,4648.837209
3826,Gas,Zentralheizung,365000.0,50.44,1.0,3.57,12163.0,1924.0,1.0,7236.320381
383,Fernwärme,Zentralheizung,154500.0,38.40,1.0,3.57,10409.0,1950.0,1.0,4023.437500
3167,Gas,Zentralheizung,209000.0,33.78,1.0,3.57,13086.0,1998.0,1.0,6187.092954
737,na,na,290000.0,32.35,1.0,3.57,13403.0,2022.0,3.0,8964.451314
736,na,na,306000.0,107.00,1.0,3.57,10709.0,1982.0,7.0,2859.813084
3830,na,na,191165.0,22.10,1.0,3.57,13088.0,2024.0,24.0,8650.000000
3166,Gas,Zentralheizung,209000.0,33.78,1.0,3.57,13086.0,1998.0,1.0,6187.092954
4371,Gas,Zentralheizung,179900.0,34.00,1.0,3.57,12435.0,1956.0,3.0,5291.176471
2610,Gas,Zentralheizung,399000.0,40.00,1.0,3.57,10245.0,2022.0,5.0,9975.000000


##### Column 'fee'

In [174]:
df_new.sort_values(by='fee', ascending=False).head(20) 

,energy,heating,price,area,rooms,fee,zipcode,construction_year,level,price_per_area
346,na,na,232000.0,60.88,3.0,7996.0,10589.0,1954.0,1.0,3810.775296
2098,Gas,Etagenheizung,817000.0,232.59,8.0,2916.0,12167.0,1905.0,4.0,3512.618771
2146,Fernwärme,Fußbodenheizung,1349000.0,130.00,4.0,48.1,10707.0,2012.0,3.0,10376.923077
4262,na,na,940000.0,133.63,4.0,33.5,10629.0,1914.0,1.0,7034.348574
2033,Öl,Zentralheizung,939000.0,167.80,5.0,33.5,10717.0,1905.0,1.0,5595.947557
2043,Gas,Zentralheizung,909000.0,201.80,4.0,32.4,13187.0,1905.0,1.0,4504.459861
2327,na,na,880000.0,93.00,3.0,31.4,10625.0,1900.0,2.0,9462.365591
1753,Gas,Etagenheizung,820000.0,120.00,4.0,29.5,12159.0,1906.0,4.0,6833.333333
2032,Öl,Zentralheizung,719000.0,114.69,2.0,25.6,10717.0,1956.0,1.0,6269.073154
2030,Öl,Zentralheizung,699000.0,125.08,4.0,24.9,10717.0,1905.0,3.0,5588.423409


The first two rows in 'fee' are outliers, I will replace these values using the median, because the mean is already skewed due to the extreme high values.

In [175]:
df_new.at[2098, 'fee'] = df_new['fee'].median()

In [176]:
df_new.at[346, 'fee'] = df_new['fee'].median()

In [177]:
# Check if the two outliers are fixed
df_new.sort_values(by='fee', ascending=False).head(20) 

,energy,heating,price,area,rooms,fee,zipcode,construction_year,level,price_per_area
2146,Fernwärme,Fußbodenheizung,1349000.0,130.00,4.0,48.1,10707.0,2012.0,3.0,10376.923077
4262,na,na,940000.0,133.63,4.0,33.5,10629.0,1914.0,1.0,7034.348574
2033,Öl,Zentralheizung,939000.0,167.80,5.0,33.5,10717.0,1905.0,1.0,5595.947557
2043,Gas,Zentralheizung,909000.0,201.80,4.0,32.4,13187.0,1905.0,1.0,4504.459861
2327,na,na,880000.0,93.00,3.0,31.4,10625.0,1900.0,2.0,9462.365591
1753,Gas,Etagenheizung,820000.0,120.00,4.0,29.5,12159.0,1906.0,4.0,6833.333333
2032,Öl,Zentralheizung,719000.0,114.69,2.0,25.6,10717.0,1956.0,1.0,6269.073154
2030,Öl,Zentralheizung,699000.0,125.08,4.0,24.9,10717.0,1905.0,3.0,5588.423409
2034,Öl,Zentralheizung,699000.0,124.49,4.0,24.9,10717.0,1905.0,2.0,5614.908828
2031,Öl,Zentralheizung,679000.0,121.18,4.0,24.2,10717.0,1905.0,1.0,5603.234857


In [178]:
df_new.sort_values(by='fee', ascending=True).head(10)  

,energy,heating,price,area,rooms,fee,zipcode,construction_year,level,price_per_area
2181,na,na,230000.0,66.78,2.0,-3.50,12043.0,1900.0,1.0,3444.144954
1607,na,na,791120.0,102.08,3.0,0.00,12587.0,2012.0,1.0,7750.000000
4078,Öl,Zentralheizung,150000.0,61.00,2.0,1.60,12103.0,1962.0,1.0,2459.016393
4102,Elektro,Etagenheizung,399000.0,105.00,3.0,1.76,10589.0,1905.0,1.0,3800.000000
4827,na,na,599000.0,113.00,3.0,1.79,12169.0,1907.0,1.0,5300.884956
2889,Gas,Zentralheizung,995000.0,132.50,5.0,1.79,10247.0,1903.0,1.0,7509.433962
4296,Gas,Etagenheizung,229000.0,74.00,2.0,1.79,12207.0,1909.0,1.0,3094.594595
4289,Gas,Etagenheizung,678950.0,127.87,4.0,1.79,10247.0,1900.0,5.0,5309.689528
3759,Öl,Zentralheizung,399000.0,78.00,2.0,1.79,12169.0,1907.0,3.0,5115.384615
4334,Gas,Etagenheizung,209000.0,67.08,3.0,1.79,12207.0,1909.0,2.0,3115.682767


The first row has a negative fee value, which is impossible. But if it is 3.5 instead of -3.5, it falls in the normal range. I will replace it with 3.5.

In [179]:
# Replace the outlier with the value 3.5
df_new.at[2181, 'fee'] = 3.50

In [180]:
df_new.sort_values(by='fee', ascending=True).head()  

,energy,heating,price,area,rooms,fee,zipcode,construction_year,level,price_per_area
1607,na,na,791120.0,102.08,3.0,0.00,12587.0,2012.0,1.0,7750.000000
4078,Öl,Zentralheizung,150000.0,61.00,2.0,1.60,12103.0,1962.0,1.0,2459.016393
4102,Elektro,Etagenheizung,399000.0,105.00,3.0,1.76,10589.0,1905.0,1.0,3800.000000
2889,Gas,Zentralheizung,995000.0,132.50,5.0,1.79,10247.0,1903.0,1.0,7509.433962
1939,Gas,Zentralheizung,950000.0,90.00,3.0,1.79,10117.0,1980.0,2.0,10555.555556


##### Column 'zipcode'

Berlin only has zipcode in the format of 1xxxxx, but the stats show a zip code 77933, it obviously is wrong.

In [181]:
df_new.sort_values(by='zipcode', ascending=False).head(20) 

,energy,heating,price,area,rooms,fee,zipcode,construction_year,level,price_per_area
1095,Gas,Zentralheizung,202000.0,48.03,2.0,1.90,77933.0,1960.0,1.0,4205.704768
325,Gas,Etagenheizung,345000.0,67.79,3.0,3.57,14199.0,1921.0,1.0,5089.246202
3446,na,na,459000.0,70.00,3.0,3.57,14199.0,1920.0,1.0,6557.142857
3497,na,na,455000.0,68.89,3.0,3.57,14199.0,1924.0,1.0,6604.732182
1970,na,na,549000.0,75.00,3.0,3.57,14199.0,1960.0,2.0,7320.000000
2203,Gas,Zentralheizung,282900.0,69.00,3.0,3.57,14199.0,1966.0,5.0,4100.000000
186,na,na,459000.0,70.00,3.0,3.57,14199.0,1956.0,1.0,6557.142857
2272,Gas,"Kamin,",990000.0,110.00,3.0,3.57,14199.0,1904.0,1.0,9000.000000
102,Fernwärme,"Fußbodenheizung, offener",1799000.0,146.50,3.0,3.57,14199.0,2022.0,1.0,12279.863481
2143,na,na,549000.0,75.00,3.0,3.57,14199.0,1965.0,2.0,7320.000000


In [182]:
# Replace the outlier with the median of zipcode
df_new.at[1095, 'zipcode'] = df_new['zipcode'].median()

In [183]:
# Check whether the replacement is successful
df_new.sort_values(by='zipcode', ascending=False).head() 

,energy,heating,price,area,rooms,fee,zipcode,construction_year,level,price_per_area
325,Gas,Etagenheizung,345000.0,67.79,3.0,3.57,14199.0,1921.0,1.0,5089.246202
3446,na,na,459000.0,70.00,3.0,3.57,14199.0,1920.0,1.0,6557.142857
1970,na,na,549000.0,75.00,3.0,3.57,14199.0,1960.0,2.0,7320.000000
186,na,na,459000.0,70.00,3.0,3.57,14199.0,1956.0,1.0,6557.142857
3349,na,na,459000.0,70.00,3.0,3.57,14199.0,1920.0,1.0,6557.142857


In [184]:
df_new.sort_values(by='zipcode', ascending=True).head(10) 

,energy,heating,price,area,rooms,fee,zipcode,construction_year,level,price_per_area
1528,Gas,Etagenheizung,639000.0,118.47,4.0,3.57,1099.0,1912.0,4.0,5393.770575
1910,na,na,590000.0,518.00,24.0,7.14,4567.0,1966.0,1.0,1138.996139
495,na,na,379000.0,44.00,1.0,2.94,10115.0,1997.0,5.0,8613.636364
3133,Gas,Zentralheizung,895000.0,98.00,4.0,3.57,10115.0,1900.0,1.0,9132.653061
661,na,na,66390.0,26.00,1.0,3.57,10115.0,1956.0,1.0,2553.461538
4295,Gas,Zentralheizung,429000.0,50.00,1.0,3.57,10115.0,1904.0,1.0,8580.000000
2476,Gas,Zentralheizung,549000.0,73.00,2.0,2.94,10115.0,1895.0,4.0,7520.547945
1630,na,na,520000.0,49.00,2.0,3.57,10115.0,2012.0,5.0,10612.244898
3545,Gas,Etagenheizung,465600.0,97.00,3.0,3.57,10115.0,1900.0,1.0,4800.000000
4280,Fernwärme,Zentralheizung,625000.0,67.00,2.0,3.57,10115.0,2014.0,3.0,9328.358209


The first two rows have wrong zipcodes. Based on my experience, the second row is likely a fraudulent listing due to a very large area and very low price. The first one could be a typo. The correct value could be 10999 or 10099, but Berlin only has the former. I will remove the second row, and replace the zipcode of the first row with 10999.

In [185]:
# Remove the second row
df_new = df_new.sort_values(by='zipcode', ascending=True).drop([1910])

In [186]:
# Replace the zipcode value in the first row with 10999
df_new.at[1528, 'zipcode'] = 10999

In [187]:
# Check whether the process was successful
df_new.sort_values(by='zipcode', ascending=True).head(10) 

,energy,heating,price,area,rooms,fee,zipcode,construction_year,level,price_per_area
4238,Gas,"Kamin,",1800000.0,168.06,6.0,3.57,10115.0,1900.0,3.0,10710.460550
1877,na,na,1200000.0,108.00,3.0,2.97,10115.0,1900.0,2.0,11111.111111
756,Gas,Zentralheizung,495000.0,64.00,3.0,7.14,10115.0,1914.0,1.0,7734.375000
660,na,na,34950.0,13.00,1.0,3.57,10115.0,1956.0,1.0,2688.461538
3014,na,na,235000.0,33.00,1.0,3.57,10115.0,1956.0,1.0,7121.212121
3013,na,na,240000.0,33.00,1.0,3.57,10115.0,1956.0,1.0,7272.727273
2587,Fernwärme,Zentralheizung,199900.0,38.02,1.0,2.38,10115.0,1996.0,1.0,5257.759074
633,na,na,175000.0,23.00,1.0,3.57,10115.0,1956.0,1.0,7608.695652
3048,Fernwärme,Zentralheizung,1100000.0,107.30,4.0,3.57,10115.0,2013.0,2.0,10251.630941
3122,Fernwärme,Fußbodenheizung,635000.0,58.70,3.0,3.57,10115.0,2014.0,4.0,10817.717206


##### Check 'construction_year'

In [188]:
df_new.sort_values(by='construction_year', ascending=False).head(10) 

,energy,heating,price,area,rooms,fee,zipcode,construction_year,level,price_per_area
2605,Fernwärme,Kf,763000.0,92.00,3.0,3.57,12557.0,2026.0,2.0,8293.478261
2606,Fernwärme,Kf,777000.0,92.80,3.0,3.57,12557.0,2026.0,2.0,8372.844828
2653,Fernwärme,Kf,481000.0,57.40,2.0,3.57,12557.0,2026.0,2.0,8379.790941
2651,Fernwärme,Kf,485000.0,56.90,2.0,3.57,12557.0,2026.0,2.0,8523.725835
2652,Fernwärme,Kf,730000.0,83.95,3.0,3.57,12557.0,2026.0,2.0,8695.652174
2265,Fernwärme,Fußbodenheizung,963038.0,107.95,3.0,3.57,12557.0,2025.0,1.0,8921.148680
1905,Fernwärme,"Fußbodenheizung,",212960.0,24.20,1.0,3.57,13088.0,2024.0,1.0,8800.000000
1317,na,na,214170.0,24.20,2.0,3.57,10405.0,2024.0,24.0,8850.000000
2830,Fernwärme,"Fußbodenheizung,",1117200.0,114.00,4.0,3.57,13088.0,2024.0,1.0,9800.000000
2531,na,na,186150.0,21.90,1.0,3.57,13088.0,2024.0,24.0,8500.000000


Even though some of the construction year lie in the future, it is a norm as properties can be on sale long before the construction is finished.

In [189]:
df_new.sort_values(by='construction_year', ascending=True).head(30) 

,energy,heating,price,area,rooms,fee,zipcode,construction_year,level,price_per_area
1973,na,na,415000.0,45.35,2.0,3.57,13156.0,1.0,2.0,9151.047409
1845,Fernwärme,Zentralheizung,2460000.0,188.00,4.0,3.57,14193.0,1.0,2.0,13085.106383
2553,na,na,290000.0,38.08,1.0,3.57,10709.0,1.0,4.0,7615.546218
4615,na,na,295000.0,67.18,3.0,3.57,13587.0,1.0,4.0,4391.187854
3379,Pellets,Zentralheizung,460000.0,111.00,2.0,3.57,10439.0,1.0,1.0,4144.144144
2517,na,na,772590.0,88.22,3.0,4.16,12203.0,1.0,1.0,8757.537973
2348,na,na,537500.0,101.00,5.0,3.57,12435.0,1.0,2.0,5321.782178
3429,Fernwärme,Fußbodenheizung,736000.0,83.95,3.0,3.57,12557.0,1.0,3.0,8767.123288
2262,na,na,369000.0,84.50,3.0,13.10,12277.0,1.0,1.0,4366.863905
1976,na,na,392000.0,41.70,2.0,3.57,13156.0,1.0,3.0,9400.479616


The first 25 rows have a wrong value in 'construction_year' per common sense. However, the differences in heating might hint that some properties are built recently, some are older. I will check the construction year distribution of the properties with floor heating.

In [190]:
df_floorheating = df_new[df_new['heating'] == 'Fußbodenheizung']

In [191]:
df_floorheating['construction_year'].value_counts(dropna=False)

construction_year
1956.0    23
2019.0    14
2022.0    12
2014.0    11
2018.0    11
2023.0    11
2021.0     7
2020.0     7
2011.0     6
2010.0     5
2015.0     5
2024.0     5
1901.0     4
1999.0     4
2016.0     4
2017.0     4
1.0        3
1910.0     3
1899.0     3
2001.0     3
2012.0     3
1936.0     3
1996.0     2
1984.0     2
1911.0     2
1897.0     2
2000.0     1
1889.0     1
1992.0     1
1874.0     1
2002.0     1
1986.0     1
1949.0     1
2025.0     1
1997.0     1
2005.0     1
1965.0     1
1940.0     1
1925.0     1
1900.0     1
2013.0     1
Name: count, dtype: int64

It looks like the properties with floor heating have very different construction years. So my assumption is wrong. I will simply replace these wrong values with random year values generated within the realistic range.

In [192]:
# Replace the construction year in the first 25 rows with a random int between 1838 and 2026.
df_new.loc[df_new['construction_year'] < 1800, 'construction_year'] = random.randint(1838, 2026)

In [193]:
# Check whether the process was successful
df_new.sort_values(by='construction_year', ascending=True).head()

,energy,heating,price,area,rooms,fee,zipcode,construction_year,level,price_per_area
4670,Fernwärme,Zentralheizung,1050000.0,104.87,3.0,3.57,10117.0,1838.0,3.0,10012.396300
4708,na,na,970000.0,98.77,3.0,3.57,10117.0,1838.0,1.0,9820.795788
4669,Fernwärme,Zentralheizung,1520000.0,132.00,3.0,3.57,10117.0,1838.0,5.0,11515.151515
4673,Fernwärme,Zentralheizung,1220000.0,118.76,3.0,3.57,10117.0,1838.0,4.0,10272.819131
4675,Fernwärme,Zentralheizung,980000.0,98.24,3.0,3.57,10117.0,1838.0,2.0,9975.570033


##### Check 'level'

In [194]:
df_new.sort_values(by='level', ascending=False).head(10) 

,energy,heating,price,area,rooms,fee,zipcode,construction_year,level,price_per_area
2257,na,na,1890000.0,213.00,7.0,2.98,12159.0,1910.0,2023.0,8873.239437
3351,na,na,204850.0,24.10,1.0,3.57,13088.0,2024.0,24.0,8500.000000
2217,Fernwärme,"Fußbodenheizung,",197120.0,22.40,1.0,3.57,13088.0,2024.0,24.0,8800.000000
2637,na,na,211750.0,24.20,3.0,3.57,13088.0,2024.0,24.0,8750.000000
3620,na,na,196000.0,22.40,1.0,3.57,13088.0,2024.0,24.0,8750.000000
1117,na,na,196875.0,22.50,3.0,3.57,13088.0,2024.0,24.0,8750.000000
2305,na,na,194625.0,22.50,3.0,3.57,13088.0,2024.0,24.0,8650.000000
2303,na,na,211925.0,24.22,3.0,3.57,13088.0,2024.0,24.0,8750.000000
3353,na,na,211750.0,24.20,1.0,3.57,13088.0,2024.0,24.0,8750.000000
966,Fernwärme,"Fußbodenheizung,",192500.0,22.00,1.0,3.57,13088.0,2024.0,24.0,8750.000000


The first row has an impossible level value of 2023. I will replace it with the average level value within the zipcode area of 12159.

In [195]:
df_zip = df_new[df_new['zipcode'] == 12159]

In [196]:
df_zip

,energy,heating,price,area,rooms,fee,zipcode,construction_year,level,price_per_area
2426,Gas,Zentralheizung,175000.0,37.00,1.0,3.57,12159.0,1983.0,1.0,4729.729730
944,Fernwärme,Zentralheizung,420000.0,66.26,2.0,3.57,12159.0,1896.0,1.0,6338.665862
2853,na,na,499000.0,112.75,4.0,3.57,12159.0,1992.0,2.0,4425.720621
1163,Öl,Zentralheizung,229000.0,47.00,1.0,3.57,12159.0,1905.0,1.0,4872.340426
2649,Fernwärme,Zentralheizung,379000.0,65.00,2.0,3.57,12159.0,1930.0,1.0,5830.769231
2257,na,na,1890000.0,213.00,7.0,2.98,12159.0,1910.0,2023.0,8873.239437
4603,na,na,250000.0,47.69,2.0,3.57,12159.0,1910.0,1.0,5242.189138
2895,Gas,Etagenheizung,1299000.0,170.99,6.0,3.57,12159.0,1908.0,1.0,7596.935493
3208,Gas,Etagenheizung,2158000.0,166.00,5.0,3.57,12159.0,1910.0,5.0,13000.000000
4002,Fernwärme,Zentralheizung,165000.0,165.40,1.0,3.57,12159.0,1956.0,1.0,997.581620


In [197]:
# Replace the outlier in df_new with the median of level in zipcode area 12159
df_new.at[2257, 'level'] = df_zip['level'].median()

In [198]:
# Check whether the replacement is successful
df_new.sort_values(by='level', ascending=False).head(10) 

,energy,heating,price,area,rooms,fee,zipcode,construction_year,level,price_per_area
599,Fernwärme,"Fußbodenheizung,",267750.0,31.50,3.0,3.57,13088.0,2024.0,24.0,8500.0
1792,na,na,197120.0,22.40,1.0,3.57,13088.0,2024.0,24.0,8800.0
2217,Fernwärme,"Fußbodenheizung,",197120.0,22.40,1.0,3.57,13088.0,2024.0,24.0,8800.0
2637,na,na,211750.0,24.20,3.0,3.57,13088.0,2024.0,24.0,8750.0
3620,na,na,196000.0,22.40,1.0,3.57,13088.0,2024.0,24.0,8750.0
1117,na,na,196875.0,22.50,3.0,3.57,13088.0,2024.0,24.0,8750.0
2305,na,na,194625.0,22.50,3.0,3.57,13088.0,2024.0,24.0,8650.0
2303,na,na,211925.0,24.22,3.0,3.57,13088.0,2024.0,24.0,8750.0
3353,na,na,211750.0,24.20,1.0,3.57,13088.0,2024.0,24.0,8750.0
966,Fernwärme,"Fußbodenheizung,",192500.0,22.00,1.0,3.57,13088.0,2024.0,24.0,8750.0


In [199]:
df_new.sort_values(by='level', ascending=True).head(10) 

,energy,heating,price,area,rooms,fee,zipcode,construction_year,level,price_per_area
650,na,na,120000.0,44.00,1.0,3.57,13403.0,1956.0,1.0,2727.272727
4865,na,na,318100.0,81.05,3.0,2.98,13187.0,1930.0,1.0,3924.737816
4810,na,na,292400.0,74.45,3.0,2.98,13187.0,1930.0,1.0,3927.468099
4078,Öl,Zentralheizung,150000.0,61.00,2.0,1.60,12103.0,1962.0,1.0,2459.016393
461,na,na,176800.0,34.00,1.0,3.57,12103.0,1957.0,1.0,5200.000000
2066,Gas,Zentralheizung,449000.0,64.00,3.0,3.57,12103.0,1926.0,1.0,7015.625000
2240,Öl,Zentralheizung,349000.0,122.07,4.0,3.57,12103.0,1980.0,1.0,2859.015319
2716,Öl,Zentralheizung,279000.0,57.00,2.0,3.57,12103.0,1900.0,1.0,4894.736842
3087,Gas,Zentralheizung,120000.0,27.00,1.0,3.57,12103.0,1930.0,1.0,4444.444444
2053,Öl,Zentralheizung,254000.0,122.07,4.0,3.57,12103.0,1980.0,1.0,2080.773327


There are no outliers in the min values of 'level'.

##### Check 'price_per_area'

In [200]:
df_new.sort_values(by='price_per_area', ascending=False).head(10) 

,energy,heating,price,area,rooms,fee,zipcode,construction_year,level,price_per_area
1575,na,na,6382140.0,72.74,3.0,3.57,10179.0,1986.0,1.0,87739.070663
2692,na,na,6382140.0,72.74,3.0,3.57,10179.0,1986.0,1.0,87739.070663
2110,Fernwärme,Kamin,15900000.0,546.00,9.0,3.57,10407.0,1875.0,1.0,29120.879121
3718,"Fußbodenheizung, offener",Zentralheizung,1650000.0,72.74,3.0,3.57,14193.0,1995.0,1.0,22683.530382
3959,na,na,4150000.0,191.40,3.0,2.38,10117.0,2015.0,4.0,21682.340648
4487,na,na,9900000.0,462.00,10.0,3.57,10117.0,2009.0,1.0,21428.571429
2232,na,na,6900000.0,326.90,9.0,3.57,10719.0,2022.0,5.0,21107.372285
781,Fernwärme,Fußbodenheizung,2495000.0,123.90,3.0,3.57,10707.0,2020.0,1.0,20137.207425
4685,Fernwärme,Fußbodenheizung,7796000.0,399.60,4.0,3.57,10117.0,2014.0,1.0,19509.509510
664,na,na,1487525.0,78.76,2.0,3.57,10625.0,2021.0,1.0,18886.808024


The first two rows have an outlier value in price_per_area, and they appear to be duplicated rows without the url column to differentiate them. Since it is difficult to estimate the realistic price_per_area as the price depends on many factors, I will just remove these two rows.

In [201]:
df_new = df_new.sort_values(by='price_per_area', ascending=False).drop([2692, 1575])

In [202]:
# Check whether the rows are dropped
df_new.sort_values(by='price_per_area', ascending=False).head(10) 

,energy,heating,price,area,rooms,fee,zipcode,construction_year,level,price_per_area
2110,Fernwärme,Kamin,15900000.0,546.00,9.0,3.57,10407.0,1875.0,1.0,29120.879121
3718,"Fußbodenheizung, offener",Zentralheizung,1650000.0,72.74,3.0,3.57,14193.0,1995.0,1.0,22683.530382
3959,na,na,4150000.0,191.40,3.0,2.38,10117.0,2015.0,4.0,21682.340648
4487,na,na,9900000.0,462.00,10.0,3.57,10117.0,2009.0,1.0,21428.571429
2232,na,na,6900000.0,326.90,9.0,3.57,10719.0,2022.0,5.0,21107.372285
781,Fernwärme,Fußbodenheizung,2495000.0,123.90,3.0,3.57,10707.0,2020.0,1.0,20137.207425
4685,Fernwärme,Fußbodenheizung,7796000.0,399.60,4.0,3.57,10117.0,2014.0,1.0,19509.509510
664,na,na,1487525.0,78.76,2.0,3.57,10625.0,2021.0,1.0,18886.808024
726,Fernwärme,Fußbodenheizung,1487500.0,79.00,2.0,3.57,10625.0,2021.0,5.0,18829.113924
3828,na,na,4100000.0,222.00,4.0,3.57,10119.0,1884.0,1.0,18468.468468


In [203]:
df_new.sort_values(by='price_per_area', ascending=True).head(50) 

,energy,heating,price,area,rooms,fee,zipcode,construction_year,level,price_per_area
4002,Fernwärme,Zentralheizung,165000.0,165.40,1.0,3.57,12159.0,1956.0,1.0,997.581620
3067,Gas,Zentralheizung,881700.0,678.23,22.0,7.14,12487.0,1996.0,1.0,1300.001474
3255,Gas,Etagenheizung,860000.0,637.22,22.0,3.57,13405.0,1956.0,4.0,1349.612379
4342,Gas,Zentralheizung,179000.0,124.23,4.0,7.14,12555.0,1909.0,5.0,1440.875795
4341,Gas,Zentralheizung,179000.0,124.23,4.0,7.14,12555.0,1909.0,5.0,1440.875795
451,na,na,109400.0,72.74,3.0,3.57,14169.0,1956.0,1.0,1503.986802
452,na,na,109400.0,72.74,3.0,3.57,13595.0,1956.0,1.0,1503.986802
4447,na,na,340000.0,225.00,6.0,3.57,13403.0,1956.0,5.0,1511.111111
2087,na,na,1100000.0,701.71,3.0,2.98,10367.0,1919.0,4.0,1567.599151
575,na,na,800000.0,498.00,3.0,3.57,13156.0,1956.0,1.0,1606.425703


- Based on my experience, a listing with a large area and unrealistically low price is highly likely a fraud. Looking at this list, I think there are many frauds. Identifying them manually would be too much effort. But through visualization, I assume it should be easy identify the frauds, because they won't blend in with authentic listings. 
- I believe it is possible to remove them but it will need a lot of investigation. The frauds likely make only a small percentage of the total listings, it could skew the data but won't skew it too much. At this point I will let it be.

##### Column 'price'¶

In [204]:
df_new.sort_values(by='price', ascending=False).head(20)  

,energy,heating,price,area,rooms,fee,zipcode,construction_year,level,price_per_area
2110,Fernwärme,Kamin,15900000.0,546.00,9.0,3.57,10407.0,1875.0,1.0,29120.879121
4487,na,na,9900000.0,462.00,10.0,3.57,10117.0,2009.0,1.0,21428.571429
4685,Fernwärme,Fußbodenheizung,7796000.0,399.60,4.0,3.57,10117.0,2014.0,1.0,19509.509510
2097,Fernwärme,Fußbodenheizung,7000000.0,450.00,10.0,3.57,10719.0,1899.0,1.0,15555.555556
2232,na,na,6900000.0,326.90,9.0,3.57,10719.0,2022.0,5.0,21107.372285
2790,na,na,6690000.0,451.00,8.0,3.57,10787.0,1956.0,1.0,14833.702882
4115,"Elektro,",Heizungsart,6500000.0,970.00,3.0,3.57,13127.0,2023.0,1.0,6701.030928
2167,na,na,5500000.0,480.00,6.0,3.57,10969.0,2008.0,1.0,11458.333333
4607,na,na,5500000.0,372.00,4.0,2.38,10119.0,2013.0,1.0,14784.946237
4582,Gas,"Fußbodenheizung, offener",5200000.0,300.00,5.0,2.20,10717.0,2023.0,1.0,17333.333333


In [205]:
df_new.sort_values(by='price', ascending=True).head(20)  

,energy,heating,price,area,rooms,fee,zipcode,construction_year,level,price_per_area
660,na,na,34950.0,13.00,1.0,3.57,10115.0,1956.0,1.0,2688.461538
661,na,na,66390.0,26.00,1.0,3.57,10115.0,1956.0,1.0,2553.461538
530,na,na,95000.0,43.10,2.0,3.57,13629.0,1963.0,1.0,2204.176334
529,na,na,95000.0,43.10,2.0,3.57,13629.0,1963.0,1.0,2204.176334
2057,Öl,Zentralheizung,98000.0,31.20,1.0,3.57,13409.0,1956.0,1.0,3141.025641
1031,Elektro,Etagenheizung,98900.0,43.05,1.0,3.57,12351.0,1956.0,2.0,2297.328688
1724,Gas,Zentralheizung,99000.0,19.00,1.0,3.57,10553.0,2018.0,1.0,5210.526316
1726,Gas,Zentralheizung,99000.0,17.41,1.0,3.57,10553.0,2018.0,1.0,5686.387134
1725,Gas,Zentralheizung,99000.0,17.41,1.0,3.57,10553.0,2018.0,1.0,5686.387134
1727,Gas,Zentralheizung,99000.0,17.50,1.0,3.57,10553.0,2018.0,1.0,5657.142857


The max and the min values of this column look fine.

##### Column 'area'

In [206]:
df_new.sort_values(by='area', ascending=False).head(20)  

,energy,heating,price,area,rooms,fee,zipcode,construction_year,level,price_per_area
4115,"Elektro,",Heizungsart,6500000.0,970.00,3.0,3.57,13127.0,2023.0,1.0,6701.030928
3004,na,na,2195800.0,821.00,26.0,3.57,10587.0,1905.0,5.0,2674.543240
2087,na,na,1100000.0,701.71,3.0,2.98,10367.0,1919.0,4.0,1567.599151
3067,Gas,Zentralheizung,881700.0,678.23,22.0,7.14,12487.0,1996.0,1.0,1300.001474
3255,Gas,Etagenheizung,860000.0,637.22,22.0,3.57,13405.0,1956.0,4.0,1349.612379
2121,Fernwärme,Zentralheizung,1839600.0,615.17,13.0,3.57,13189.0,1910.0,1.0,2990.392900
4903,na,na,4700000.0,600.00,7.0,3.57,10629.0,2004.0,1.0,7833.333333
4776,Gas,"Kamin,",2250000.0,570.00,15.0,7.14,12047.0,1914.0,5.0,3947.368421
4766,Öl,Zentralheizung,1550000.0,558.06,18.0,3.57,13589.0,1964.0,1.0,2777.479124
1302,Gas,Etagenheizung,1322000.0,550.82,14.0,5.95,12049.0,1956.0,1.0,2400.058095


In [207]:
df_new.sort_values(by='price', ascending=True).head(20)  

,energy,heating,price,area,rooms,fee,zipcode,construction_year,level,price_per_area
660,na,na,34950.0,13.00,1.0,3.57,10115.0,1956.0,1.0,2688.461538
661,na,na,66390.0,26.00,1.0,3.57,10115.0,1956.0,1.0,2553.461538
530,na,na,95000.0,43.10,2.0,3.57,13629.0,1963.0,1.0,2204.176334
529,na,na,95000.0,43.10,2.0,3.57,13629.0,1963.0,1.0,2204.176334
2057,Öl,Zentralheizung,98000.0,31.20,1.0,3.57,13409.0,1956.0,1.0,3141.025641
1031,Elektro,Etagenheizung,98900.0,43.05,1.0,3.57,12351.0,1956.0,2.0,2297.328688
1724,Gas,Zentralheizung,99000.0,19.00,1.0,3.57,10553.0,2018.0,1.0,5210.526316
1726,Gas,Zentralheizung,99000.0,17.41,1.0,3.57,10553.0,2018.0,1.0,5686.387134
1725,Gas,Zentralheizung,99000.0,17.41,1.0,3.57,10553.0,2018.0,1.0,5686.387134
1727,Gas,Zentralheizung,99000.0,17.50,1.0,3.57,10553.0,2018.0,1.0,5657.142857


The max and the min values of this column look fine.

### 2.5 Export the dataframe to pickle 

In [208]:
# Export the new dataframe
df_new.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'df_cleaned.pkl'))

# 3. Data profile (Step 7)

In [209]:
df_new.describe()

,price,area,rooms,fee,zipcode,construction_year,level,price_per_area
count,4.937000e+03,4937.000000,4937.000000,4937.000000,4937.000000,4937.000000,4937.000000,4937.000000
mean,5.619536e+05,84.981057,2.797650,3.685817,11945.913713,1953.642495,2.994126,6354.012572
std,5.939564e+05,58.016705,1.530276,1.579564,1316.975885,45.138612,4.830155,2472.453375
min,3.495000e+04,13.000000,1.000000,0.000000,10115.000000,1838.000000,1.000000,997.581620
25%,2.590000e+05,54.040000,2.000000,3.570000,10589.000000,1910.000000,1.000000,4538.333333
50%,3.899000e+05,72.740000,3.000000,3.570000,12161.000000,1956.000000,1.000000,5843.852267
75%,6.690000e+05,101.000000,3.000000,3.570000,13088.000000,1994.000000,3.000000,8055.555556
max,1.590000e+07,970.000000,26.000000,48.100000,14199.000000,2026.000000,24.000000,29120.879121


In [210]:
df_new.shape

(4937, 10)